In [127]:
import pandas as pd
import numpy as np


from sklearn.metrics import make_scorer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score, make_scorer, mean_absolute_error, mean_squared_error
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import make_pipeline

from wordcloud import WordCloud
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

import kagglehub
import tqdm as notebook_tqdm

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sara\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [128]:
final_model = pd.read_csv("data_preprocessed.csv")

In [129]:
final_model.describe()

,Rating,min_salary,max_salary,avg_salary,Company Age,Seniority_code,contains_requirements,contains_team,contains_reporting,contains_information,contains_management,contains_skills,Company Offers,num_competitors,Easy_Apply
count,2252.000000,2252.000000,2252.000000,2252.000000,2252.000000,2252.000000,2252.000000,2252.000000,2252.000000,2252.000000,2252.000000,2252.000000,2252.000000,2252.000000,2252.000000
mean,3.729287,54.266874,89.979130,72.123002,39.254440,1.204263,0.595027,0.734902,0.500888,0.527087,0.634547,0.786856,4.193606,0.609236,0.035524
std,0.643908,19.575375,29.315605,23.600734,42.032931,0.602836,0.490996,0.441483,0.500110,0.499377,0.481664,0.409619,9.480453,1.148295,0.185141
min,1.000000,24.000000,38.000000,33.500000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
25%,3.300000,41.000000,70.000000,58.000000,17.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000
50%,3.700000,50.000000,87.000000,69.000000,24.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000
75%,4.100000,64.000000,104.000000,80.500000,40.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,3.000000,0.000000,0.000000
max,5.000000,113.000000,190.000000,150.000000,343.000000,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,58.000000,4.000000,1.000000


## 3. MÈTRIQUES DE SELECCIÓ

En aquest projecte de predicció de sous, no hem d'escollir una mètrica de selecció concreta. Farem servir mètriques que analitzin l'error comès a les prediccions i aixi poder escollir el model que cometi menys error, és a dir, el que predigui millor el sou de les ofertes de treball.

In [130]:
X = final_model.drop(columns=['avg_salary', 'min_salary', 'max_salary'])
y = final_model[['avg_salary','min_salary','max_salary']]

Per avaluar el rendiment dels models de predicció de salaris farem servir les mètriques: 
- MAE: indica l’error mitjà absolut i permet interpretar directament quants diners s’equivoca el model de mitjana del sou.
- RMSE: penalitza més els errors grans i és especialment útil quan volem evitar prediccions molt allunyades del valor real.
- R²: mesura quina proporció de la variabilitat del salari és capaç d’explicar el model.

Considerem que una combinació de MAE i RMSE és la més adequada per comparar models, ja que permet quantificar l’error real i, alhora, detectar si el model comet prediccions extremes, mentre que el R² serveix de suport per entendre el grau d’ajust global.

## 4. MODELITZACIÓ

**4.1 Salary mínim**

Primer cal dividir les dades en conjunt d’entrenament i test (tipus 80%-20% o 70%-30%):

In [131]:
X_train_lower, X_test_lower, y_train_lower, y_test_lower = \
    train_test_split(final_model,final_model['min_salary'].values,test_size=0.3, random_state=0)


In [132]:
y_train_lower.mean(), y_test_lower.mean()

(np.float64(54.75761421319797), np.float64(53.12278106508876))

**Normalització de dades**


Normalitzem les dades una vegada fet el split del conjunt d'entrenament i de test per minimitzar la pèrdua d'informació.

In [133]:
# Definir les columnes numèriques contínues a normalitzar
num_cols = ['Company Age', 'Company Offers', 'num_competitors']

scaler = StandardScaler()
X_train_lower[num_cols] = scaler.fit_transform(X_train_lower[num_cols])
X_test_lower[num_cols] = scaler.transform(X_test_lower[num_cols])

# Comprovació
X_train_lower.describe()

,Rating,min_salary,max_salary,avg_salary,Company Age,Seniority_code,contains_requirements,contains_team,contains_reporting,contains_information,contains_management,contains_skills,Company Offers,num_competitors,Easy_Apply
count,1576.000000,1576.000000,1576.000000,1576.000000,1.576000e+03,1576.000000,1576.000000,1576.000000,1576.000000,1576.000000,1576.000000,1576.000000,1.576000e+03,1.576000e+03,1576.000000
mean,3.718051,54.757614,90.715102,72.736358,6.762780e-17,1.216371,0.600254,0.730330,0.501904,0.538071,0.642766,0.785533,2.254260e-18,3.155964e-17,0.031726
std,0.637262,19.626988,29.439430,23.676064,1.000317e+00,0.612680,0.490002,0.443929,0.500155,0.498707,0.479337,0.410582,1.000317e+00,1.000317e+00,0.175325
min,1.000000,24.000000,38.000000,33.500000,-7.823817e-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-3.370486e-01,-5.292101e-01,0.000000
25%,3.300000,42.000000,72.000000,59.000000,-5.292677e-01,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,-3.370486e-01,-5.292101e-01,0.000000
50%,3.700000,51.000000,87.000000,69.500000,-3.681951e-01,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-3.370486e-01,-5.292101e-01,0.000000
75%,4.100000,64.250000,104.000000,80.625000,-2.920097e-05,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-1.249973e-01,-5.292101e-01,0.000000
max,5.000000,113.000000,190.000000,150.000000,6.972112e+00,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,5.706414e+00,2.090796e+00,1.000000


**Selecció de models**

In [134]:
# models a provar
models = {
    'LinearRegression': LinearRegression(),
    'Ridge': Ridge(alpha=1.0, random_state=42),
    'Lasso': Lasso(alpha=0.1, random_state=42),
    'RandomForest': RandomForestRegressor(n_estimators=100, random_state=42),
    'GradientBoosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
    'KNN': KNeighborsRegressor(n_neighbors=5)
}

# Configurar cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Funcions de scoring
mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)
rmse_scorer = make_scorer(lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred)), greater_is_better=False)
r2_scorer = make_scorer(r2_score)

# resultats
results_list = []

# Iterar sobre models
for name, model in models.items():
    mae_scores = cross_val_score(model, X_train_lower, y_train_lower, cv=kf, scoring=mae_scorer)
    rmse_scores = cross_val_score(model, X_train_lower, y_train_lower, cv=kf, scoring=rmse_scorer)
    r2_scores = cross_val_score(model, X_train_lower, y_train_lower, cv=kf, scoring=r2_scorer)
    
    # Agregar a la lista
    results_list.append({
        'Model': name,
        'MAE_mean': -mae_scores.mean(),  # Negatiu perque sklearn retorna negatiu
        'MAE_std': mae_scores.std(),
        'RMSE_mean': -rmse_scores.mean(),
        'RMSE_std': rmse_scores.std(),
        'R2_mean': r2_scores.mean(),
        'R2_std': r2_scores.std()
    })

results = pd.DataFrame(results_list)


results = results.sort_values(by='MAE_mean')
print(results)

              Model      MAE_mean       MAE_std     RMSE_mean      RMSE_std  \
0  LinearRegression  1.566907e-14  5.858111e-15  2.403140e-14  7.505043e-15   
1             Ridge  6.943805e-05  5.471851e-06  9.630970e-05  9.529907e-06   
4  GradientBoosting  3.535572e-03  2.269891e-03  1.995868e-02  3.023723e-02   
2             Lasso  3.564767e-03  1.551079e-04  4.562256e-03  1.106859e-04   
3      RandomForest  6.068274e-03  3.210424e-03  5.147575e-02  3.678123e-02   
5               KNN  6.739321e-02  1.739033e-02  2.467619e-01  6.093011e-02   

    R2_mean        R2_std  
0  1.000000  0.000000e+00  
1  1.000000  5.281793e-12  
4  0.999997  6.292300e-06  
2  1.000000  1.414031e-09  
3  0.999990  1.309437e-05  
5  0.999832  8.634394e-05  


**Selecció d'hiperparàmetres**

Definim el grid de paràmetres:

In [135]:
param_grids = {
    "KNN": {
        "model": KNeighborsRegressor(),
        "params": {
            "n_neighbors": [3, 5, 7, 9, 15],
            "weights": ["uniform", "distance"],
            "p": [1, 2]  # Manhattan o Euclidiana
        }
    },

    "RandomForest": {
        "model": RandomForestRegressor(random_state=42),
        "params": {
            "n_estimators": [100, 200, 300],
            "max_depth": [None, 10, 20, 30],
            "min_samples_split": [2, 5, 10],
            "min_samples_leaf": [1, 2, 4]
        }
    },

    "GradientBoosting": {
        "model": GradientBoostingRegressor(random_state=42),
        "params": {
            "n_estimators": [100, 200, 300],
            "learning_rate": [0.01, 0.05, 0.1],
            "max_depth": [2, 3, 4],
            "subsample": [0.6, 0.8, 1.0]
        }
    },

    "Ridge": {
        "model": Ridge(),
        "params": {
            "alpha": [0.01, 0.1, 1, 10, 100]
        }
    },

    "Lasso": {
        "model": Lasso(max_iter=5000),
        "params": {
            "alpha": [0.0001, 0.001, 0.01, 0.1, 1]
        }
    }
}

Executar gridsearch pel model:

In [136]:
best_models = {}

for name, cfg in param_grids.items():
    print(f"\n Optimitzant {name}...\n")
    
    grid = GridSearchCV(
        estimator=cfg["model"],
        param_grid=cfg["params"],
        cv=5,
        scoring="neg_mean_absolute_error",
        n_jobs=-1
    )

    grid.fit(X_train_lower, y_train_lower)

    best_models[name] = grid.best_estimator_

    print(f"Millor score MAE: {-grid.best_score_:.2f}")
    print(f"Millors hiperparàmetres: {grid.best_params_}")


 Optimitzant KNN...

Millor score MAE: 0.04
Millors hiperparàmetres: {'n_neighbors': 3, 'p': 1, 'weights': 'distance'}

 Optimitzant RandomForest...

Millor score MAE: 0.01
Millors hiperparàmetres: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}

 Optimitzant GradientBoosting...

Millor score MAE: 0.00
Millors hiperparàmetres: {'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 300, 'subsample': 1.0}

 Optimitzant Ridge...

Millor score MAE: 0.00
Millors hiperparàmetres: {'alpha': 0.01}

 Optimitzant Lasso...

Millor score MAE: 0.00
Millors hiperparàmetres: {'alpha': 0.0001}


Veiem que alguns hiperparametres escollits estant a un extrem del grid així que explorem un nou espai d'hiperparàmetres en aquells casos per veure si aconseguim millorar encara més el model.

In [137]:
param_grids_refined = {
    "KNN": {
        "model": KNeighborsRegressor(),
        "params": {
            "n_neighbors": [15, 20, 25, 30, 40, 50],
            "weights": ["uniform", "distance"],
            "p": [1, 2]
        }
    },

    "RandomForest": {
        "model": RandomForestRegressor(random_state=42),
        "params": {
            "n_estimators": [200, 300, 400],
            "max_depth": [5, 10, 15, 20],
            "min_samples_split": [2, 3, 4, 6],
            "min_samples_leaf": [2, 4, 6]
        }
    },

    "GradientBoosting": {
        "model": GradientBoostingRegressor(random_state=42),
        "params": {
            "n_estimators": [300, 400, 500],
            "learning_rate": [0.005, 0.01, 0.02],
            "max_depth": [4, 5, 6],
            "subsample": [0.6, 0.8, 1.0]
        }
    },

    "Ridge": {
        "model": Ridge(),
        "params": {
            "alpha": [100, 200, 300, 500, 800, 1000]
        }
    },

    "Lasso": {
        "model": Lasso(max_iter=8000),
        "params": {
            "alpha": [1, 2, 5, 10, 20, 50]
        }
    }
}

In [138]:
best_models_refined = {}

for name, cfg in param_grids_refined.items():
    print(f"\n Optimitzant {name}...\n")

    grid = GridSearchCV(
        estimator=cfg["model"],
        param_grid=cfg["params"],
        cv=5,
        scoring="neg_mean_absolute_error",
        n_jobs=-1,
    )

    grid.fit(X_train_lower, y_train_lower)

    best_models_refined[name] = grid.best_estimator_

    print(f"Millor MAE: {-grid.best_score_:.2f}")
    print(f"Millors hiperparàmetres: {grid.best_params_}")


 Optimitzant KNN...

Millor MAE: 0.20
Millors hiperparàmetres: {'n_neighbors': 15, 'p': 2, 'weights': 'distance'}

 Optimitzant RandomForest...

Millor MAE: 0.01
Millors hiperparàmetres: {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 400}

 Optimitzant GradientBoosting...

Millor MAE: 0.00
Millors hiperparàmetres: {'learning_rate': 0.02, 'max_depth': 4, 'n_estimators': 500, 'subsample': 1.0}

 Optimitzant Ridge...

Millor MAE: 0.01
Millors hiperparàmetres: {'alpha': 100}

 Optimitzant Lasso...

Millor MAE: 0.04
Millors hiperparàmetres: {'alpha': 1}


In [139]:
def get_mae(model):
    mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)
    scores = cross_val_score(model, X_train_lower, y_train_lower, cv=5, scoring=mae_scorer)
    return -scores.mean()

rows = []

# Afegir models de la 1a optimització
for name, model in best_models.items():
    rows.append({
        "Model": f"{name} (Round 1)",
        "MAE": get_mae(model),
        "Hiperparàmetres": model.get_params()
    })

"""# Afegir models de la 2a optimització
for name, model in best_models_refined.items():
    rows.append({
        "Model": f"{name} (Round 2)",
        "MAE": get_mae(model),
        "Hiperparàmetres": model.get_params()
    })"""

results_table = pd.DataFrame(rows)
results_table

,Model,MAE,Hiperparàmetres
0,KNN (Round 1),4.042856e-02,"{'algorithm': 'auto', 'leaf_size': 30, 'metric..."
1,RandomForest (Round 1),5.321991e-03,"{'bootstrap': True, 'ccp_alpha': 0.0, 'criteri..."
2,GradientBoosting (Round 1),6.617998e-06,"{'alpha': 0.9, 'ccp_alpha': 0.0, 'criterion': ..."
3,Ridge (Round 1),7.315267e-07,"{'alpha': 0.01, 'copy_X': True, 'fit_intercept..."
4,Lasso (Round 1),3.636952e-04,"{'alpha': 0.0001, 'copy_X': True, 'fit_interce..."


Veiem que el Ridge obté el MAE més baix.

Veiem que el segon model que presenta millors resultats és el Gradient Boosting Regressor però tenint certa distància amb el Random Forest que per tant és el que escollim com a millor model per predir el salari estimat.

A continuació amb aquest model entrenarem de manera final sobre tot el conjunt d'entrenament i després avaluarem sobre el conjunt de test.

In [140]:
model_entries = [r for r in rows]

model_results = [
    (entry["Model"], entry["MAE"], entry["Hiperparàmetres"])
    for entry in model_entries
]

# Seleccionar el model amb MAE més baix
best_model_name, best_model_mae, best_model_params = min(model_results, key=lambda x: x[1])

print("Millor model:", best_model_name)
print("MAE:", best_model_mae)
best_model_params

Millor model: Ridge (Round 1)
MAE: 7.315267014035013e-07


{'alpha': 0.01,
 'copy_X': True,
 'fit_intercept': True,
 'max_iter': None,
 'positive': False,
 'random_state': None,
 'solver': 'auto',
 'tol': 0.0001}

**Entrenament model seleccionat**

Entrenarem el model unicament pel valor mig del salari i per verificar si és correcte, comprovarem si aquest valor es troba dins de l'interval real de salari mínim i màxim.

Fem això perquè el valor mig del salari resumeix de manera senzilla la informació salarial de cada empresa i ens permet entrenar el model amb una sola variable contínua, evitant la complexitat de tenir múltiples valors per cada catacterística. A més, verificar que el valor predit es troba dins de l’interval real de salari mínim i màxim ens assegura que el model respecta les restriccions reals del salari i evita prediccions fora de rang.

In [141]:
from sklearn.linear_model import Ridge
best_ridge = Ridge(
    alpha=0.01,
    copy_X=True,
    fit_intercept=True,
    max_iter=None,
    positive=False,
    random_state=None,
    solver='auto',
    tol=0.0001
)
best_ridge.fit(X_train_lower, y_train_lower)


,alpha,0.01
,fit_intercept,True
,copy_X,True
,max_iter,None
,tol,0.0001
,solver,'auto'
,positive,False
,random_state,None


**Prova conjunt de test**

In [142]:
y_pred = best_ridge.predict(X_test_lower)

In [143]:
mae = mean_absolute_error(y_test_lower, y_pred)
rmse = np.sqrt(mean_squared_error(y_test_lower, y_pred))
r2 = r2_score(y_test_lower, y_pred)

In [144]:
print("Resultats del model:")
print(f"MAE  : {mae}")
print(f"RMSE : {rmse}")
print(f"R²   : {r2}")

Resultats del model:
MAE  : 5.54121392095337e-07
RMSE : 7.745820713538344e-07
R²   : 0.9999999999999984


**4.2 Max Salary**


Repetim tot el procediment per max salary:

In [145]:
X_train_high, X_test_high, y_train_high, y_test_high = \
    train_test_split(final_model,final_model['max_salary'].values,test_size=0.3, random_state=0)


In [146]:
y_train_high.mean(),y_test_high.mean()

(np.float64(90.71510152284264), np.float64(88.26331360946746))

In [147]:
# Definir les columnes numèriques contínues a normalitzar
num_cols = ['Company Age',
            'Company Offers', 'num_competitors']

scaler = StandardScaler()
X_train_high[num_cols] = scaler.fit_transform(X_train_high[num_cols])

# Comprovació
X_train_high.describe()

,Rating,min_salary,max_salary,avg_salary,Company Age,Seniority_code,contains_requirements,contains_team,contains_reporting,contains_information,contains_management,contains_skills,Company Offers,num_competitors,Easy_Apply
count,1576.000000,1576.000000,1576.000000,1576.000000,1.576000e+03,1576.000000,1576.000000,1576.000000,1576.000000,1576.000000,1576.000000,1576.000000,1.576000e+03,1.576000e+03,1576.000000
mean,3.718051,54.757614,90.715102,72.736358,6.762780e-17,1.216371,0.600254,0.730330,0.501904,0.538071,0.642766,0.785533,2.254260e-18,3.155964e-17,0.031726
std,0.637262,19.626988,29.439430,23.676064,1.000317e+00,0.612680,0.490002,0.443929,0.500155,0.498707,0.479337,0.410582,1.000317e+00,1.000317e+00,0.175325
min,1.000000,24.000000,38.000000,33.500000,-7.823817e-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-3.370486e-01,-5.292101e-01,0.000000
25%,3.300000,42.000000,72.000000,59.000000,-5.292677e-01,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,-3.370486e-01,-5.292101e-01,0.000000
50%,3.700000,51.000000,87.000000,69.500000,-3.681951e-01,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-3.370486e-01,-5.292101e-01,0.000000
75%,4.100000,64.250000,104.000000,80.625000,-2.920097e-05,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-1.249973e-01,-5.292101e-01,0.000000
max,5.000000,113.000000,190.000000,150.000000,6.972112e+00,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,5.706414e+00,2.090796e+00,1.000000


In [148]:
# models a provar
models = {
    'LinearRegression': LinearRegression(),
    'Ridge': Ridge(alpha=1.0, random_state=42),
    'Lasso': Lasso(alpha=0.1, random_state=42),
    'RandomForest': RandomForestRegressor(n_estimators=100, random_state=42),
    'GradientBoosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
    'KNN': KNeighborsRegressor(n_neighbors=5)
}

# Configurar cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Funcions de scoring
mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)
rmse_scorer = make_scorer(lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred)), greater_is_better=False)
r2_scorer = make_scorer(r2_score)

# resultats
results_list = []

# Iterar sobre models
for name, model in models.items():
    mae_scores = cross_val_score(model, X_train_high, y_train_high, cv=kf, scoring=mae_scorer)
    rmse_scores = cross_val_score(model, X_train_high, y_train_high, cv=kf, scoring=rmse_scorer)
    r2_scores = cross_val_score(model, X_train_high, y_train_high, cv=kf, scoring=r2_scorer)
    
    # Agregar a la lista
    results_list.append({
        'Model': name,
        'MAE_mean': -mae_scores.mean(),  # Negatiu perque sklearn retorna negatiu
        'MAE_std': mae_scores.std(),
        'RMSE_mean': -rmse_scores.mean(),
        'RMSE_std': rmse_scores.std(),
        'R2_mean': r2_scores.mean(),
        'R2_std': r2_scores.std()
    })

results = pd.DataFrame(results_list)


results = results.sort_values(by='MAE_mean')
print(results)

c:\Users\sara\Desktop\APC\.venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.816e+02, tolerance: 1.122e+02
  model = cd_fast.enet_coordinate_descent(
c:\Users\sara\Desktop\APC\.venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.819e+02, tolerance: 1.111e+02
  model = cd_fast.enet_coordinate_descent(
c:\Users\sara\Desktop\APC\.venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap

              Model      MAE_mean       MAE_std     RMSE_mean      RMSE_std  \
0  LinearRegression  1.977246e-14  1.061858e-14  2.700883e-14  1.161955e-14   
1             Ridge  5.619124e-05  3.004875e-06  7.245989e-05  4.924288e-06   
3      RandomForest  8.052361e-03  3.640235e-03  5.145774e-02  2.044524e-02   
2             Lasso  1.156257e-02  5.792560e-04  1.501942e-02  7.542054e-04   
4  GradientBoosting  1.624642e-02  3.484201e-03  4.052761e-02  2.523852e-02   
5               KNN  6.841390e-02  1.119386e-02  2.458025e-01  3.532505e-02   

    R2_mean        R2_std  
0  1.000000  0.000000e+00  
1  1.000000  1.360563e-12  
3  0.999997  2.110724e-06  
2  1.000000  3.543431e-08  
4  0.999997  3.896862e-06  
5  0.999929  1.897154e-05  


In [149]:
best_models = {}

for name, cfg in param_grids.items():
    print(f"\n Optimitzant {name}...\n")
    
    grid = GridSearchCV(
        estimator=cfg["model"],
        param_grid=cfg["params"],
        cv=5,
        scoring="neg_mean_absolute_error",
        n_jobs=-1
    )

    grid.fit(X_train_high, y_train_high)

    best_models[name] = grid.best_estimator_

    print(f"Millor score MAE: {-grid.best_score_:.2f}")
    print(f"Millors hiperparàmetres: {grid.best_params_}")


 Optimitzant KNN...

Millor score MAE: 0.04
Millors hiperparàmetres: {'n_neighbors': 3, 'p': 2, 'weights': 'distance'}

 Optimitzant RandomForest...

Millor score MAE: 0.01
Millors hiperparàmetres: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 300}

 Optimitzant GradientBoosting...

Millor score MAE: 0.00
Millors hiperparàmetres: {'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 300, 'subsample': 1.0}

 Optimitzant Ridge...

Millor score MAE: 0.00
Millors hiperparàmetres: {'alpha': 0.01}

 Optimitzant Lasso...

Millor score MAE: 0.00
Millors hiperparàmetres: {'alpha': 0.0001}


In [150]:
def get_mae2(model):
    mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)
    scores = cross_val_score(model, X_train_high, y_train_high, cv=5, scoring=mae_scorer)
    return -scores.mean()

rows = []

# Afegir models de la 1a optimització
for name, model in best_models.items():
    rows.append({
        "Model": f"{name} (Round 1)",
        "MAE": get_mae(model),
        "Hiperparàmetres": model.get_params()
    })

# Afegir models de la 2a optimització
for name, model in best_models_refined.items():
    rows.append({
        "Model": f"{name} (Round 2)",
        "MAE": get_mae(model),
        "Hiperparàmetres": model.get_params()
    })

results_table = pd.DataFrame(rows)
results_table

,Model,MAE,Hiperparàmetres
0,KNN (Round 1),4.043811e-02,"{'algorithm': 'auto', 'leaf_size': 30, 'metric..."
1,RandomForest (Round 1),5.427100e-03,"{'bootstrap': True, 'ccp_alpha': 0.0, 'criteri..."
2,GradientBoosting (Round 1),7.274481e-06,"{'alpha': 0.9, 'ccp_alpha': 0.0, 'criterion': ..."
3,Ridge (Round 1),7.315267e-07,"{'alpha': 0.01, 'copy_X': True, 'fit_intercept..."
4,Lasso (Round 1),3.636952e-04,"{'alpha': 0.0001, 'copy_X': True, 'fit_interce..."
5,KNN (Round 2),2.045172e-01,"{'algorithm': 'auto', 'leaf_size': 30, 'metric..."
6,RandomForest (Round 2),6.625794e-03,"{'bootstrap': True, 'ccp_alpha': 0.0, 'criteri..."
7,GradientBoosting (Round 2),6.607328e-04,"{'alpha': 0.9, 'ccp_alpha': 0.0, 'criterion': ..."
8,Ridge (Round 2),5.707399e-03,"{'alpha': 100, 'copy_X': True, 'fit_intercept'..."
9,Lasso (Round 2),4.097030e-02,"{'alpha': 1, 'copy_X': True, 'fit_intercept': ..."


In [151]:
model_entries = [r for r in rows]

model_results = [
    (entry["Model"], entry["MAE"], entry["Hiperparàmetres"])
    for entry in model_entries
]

# Seleccionar el model amb MAE més baix
best_model_name, best_model_mae, best_model_params = min(model_results, key=lambda x: x[1])

print("Millor model:", best_model_name)
print("MAE:", best_model_mae)
best_model_params

Millor model: Ridge (Round 1)
MAE: 7.315267014035013e-07


{'alpha': 0.01,
 'copy_X': True,
 'fit_intercept': True,
 'max_iter': None,
 'positive': False,
 'random_state': None,
 'solver': 'auto',
 'tol': 0.0001}

In [152]:
from sklearn.linear_model import Ridge
best_ridge = Ridge(
    alpha=0.01,
    copy_X=True,
    fit_intercept=True,
    max_iter=None,
    positive=False,
    random_state=None,
    solver='auto',
    tol=0.0001
)
best_ridge.fit(X_train_high, y_train_high)


,alpha,0.01
,fit_intercept,True
,copy_X,True
,max_iter,None
,tol,0.0001
,solver,'auto'
,positive,False
,random_state,None


In [153]:
y_pred_high = best_ridge.predict(X_test_high)

In [154]:
mae = mean_absolute_error(y_test_high, y_pred_high)
rmse = np.sqrt(mean_squared_error(y_test_high, y_pred_high))
r2 = r2_score(y_test_high, y_pred_high)

In [155]:
print("Resultats del model:")
print(f"MAE  : {mae}")
print(f"RMSE : {rmse}")
print(f"R²   : {r2}")

Resultats del model:
MAE  : 7.460970366074042e-07
RMSE : 1.0458207457588661e-06
R²   : 0.9999999999999987


In [156]:
df_export = pd.DataFrame({
    "real_min_salary": y_test_lower,
    "pred_min_salary": y_pred,
    "real_max_salary": y_test_high,
    "pred_max_salary": y_pred_high
})

file_name = "data_predicted.csv"
df_export.to_csv(file_name, index=False)
df_export.head()


,real_min_salary,pred_min_salary,real_max_salary,pred_max_salary
0,45,44.999999,78,78.000001
1,26,26.000000,47,47.000001
2,64,64.000000,113,113.000000
3,97,96.999999,129,129.000000
4,37,37.000000,68,68.000000
